<a href="https://colab.research.google.com/github/velusrinathmr/agenticai/blob/main/Google_ADK_Multi_agent_TrainingAssess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation of Google ADK and other dependencies

In [1]:
pip install google-adk -q

# ENV variables

## Imports

In [2]:
import os

## Env variable allocation

In [3]:
from google.colab import userdata
try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
except:
  print("No Google API Key found")
  GOOGLE_API_KEY = input("Enter Google API Key manually")

In [4]:
GOOGLE_GENAI_USE_VERTEXAI = "FALSE"

# Agent Development

## Imports

In [5]:
from google.adk.agents import Agent, LlmAgent, LoopAgent, SequentialAgent

## Tools creation

### Flight search tool

In [6]:
def search_flights(destination: str):

    flights = []

    if "london" in destination.lower():
        flights.append({"airline": "AirIndia", "price": 25000, "departure_time": "08:00", "arrival_time": "23:30"})
        flights.append({"airline": "Swiss Airlines", "price": 60000, "departure_time": "10:00", "arrival_time": "03:45"})
    elif "new york" in destination.lower():
        flights.append({"airline": "Air India", "price": 45000, "departure_time": "09:00", "arrival_time": "01:00"})
        flights.append({"airline": "Qatar Airways", "price": 50000, "departure_time": "11:00", "arrival_time": "07:30"})
    else:
        flights.append({"airline": "Emirates", "price": 30000, "departure_time": "07:00", "arrival_time": "12:00"})
    return flights

### Hotel search tool

In [7]:
def google_hotels(location: str, check_in_date: str, check_out_date: str, min_stars: int = 3):

    hotels = []

    if "london" in location.lower():
        hotels.append({"name": "Grand Stay Inn", "price_per_night": 150, "stars": 4, "amenities": ["Wifi", "Breakfast"]})
        hotels.append({"name": "Budget Lodge", "price_per_night": 80, "stars": 3, "amenities": ["Wifi"]})
    elif "new york" in location.lower():
        hotels.append({"name": "City View Suites", "price_per_night": 250, "stars": 4.5, "amenities": ["Wifi", "Gym", "Pool"]})
        hotels.append({"name": "Downtown Hostel", "price_per_night": 70, "stars": 2.5, "amenities": ["Shared Kitchen"]})
    else:
        hotels.append({"name": "Cozy Cottage", "price_per_night": 120, "stars": 4, "amenities": ["Wifi", "Parking"]})
    return hotels

### Activites search

In [8]:
def search_activities(location, interests):

    activities = []

    if "london" in location.lower():
        activities.append({"name": "British Museum Tour", "type": "Culture", "price": 0})
        activities.append({"name": "Thames River Cruise", "type": "Sightseeing", "price": 30})
        if interests and "food" in [i.lower() for i in interests]:
            activities.append({"name": "Borough Market Food Tour", "type": "Food", "price": 75})
    elif "new york" in location.lower():
        activities.append({"name": "Central Park Bike Rental", "type": "Outdoors", "price": 25})
        activities.append({"name": "Broadway Show Tickets", "type": "Entertainment", "price": 120})
    else:
        activities.append({"name": "Central Park", "type": "Outdoors", "price": 25})
    return activities

## Agent creation

In [9]:
llm_model = "gemini-1.5-flash"

In [10]:
flight_agent = LlmAgent(
    name="FlightAgent",
    model=llm_model,
    description="An agent specialized in finding flight information.",
    instruction=(
        "You are an expert in finding flights. Use the 'search_flights' tool to find flight options. "
        "Always ask for destination, and departure date. If return date is missing for a round trip, ask for it. "
        "Present flight options clearly, including airline, price, and times."
        "you will only use search_flights tool to look for flights."
    ),
    tools=[search_flights],
)

In [11]:
hotel_agent = LlmAgent(
    name="HotelAgent",
    model=llm_model,
    description="An agent specialized in finding hotel accommodations.",
    instruction="You are an expert in finding hotels. You will only use the 'google_hotels' tool to find hotel options.",
    tools=[google_hotels],
)

In [12]:
activity_agent = LlmAgent(
    name="ActivityAgent",
    model=llm_model,
    description="An agent specialized in suggesting activities and attractions.",
    instruction="you will only use 'search_activities' tool to suggest local activities.",
    tools=[search_activities],
)

In [13]:
travel_coordinator = LlmAgent(
    name="TravelCoordinatorAgent",
    model=llm_model,
    description="Main coordinator for travel planning tasks.",
    instruction=(
        "Understand user travel goals and coordinate sub-agents (FlightAgent, HotelAgent, ActivityAgent). "
        "You can also ask for user's name and date details to ensure accurate information."
    ),
    sub_agents=[flight_agent, hotel_agent, activity_agent],
)

# Session and Runner setup

## Session setup

In [14]:
from google.adk.sessions import InMemorySessionService

In [15]:
APP_NAME="travel_planner_app"
USER_ID="user_1"
SESSION_ID="session_001"

In [16]:
session_service=InMemorySessionService()

session = await session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID,
)

## Runner setup

In [17]:
from google.adk.runners import Runner

In [18]:
runner=Runner(
    agent=travel_coordinator,
    app_name=APP_NAME,
    session_service=session_service
)

# Testing Agent Interaction

In [19]:
import asyncio
from google.genai import types

In [20]:
from IPython.display import display, Markdown

In [21]:
prompt = "Plan a trip to Paris for 5 days"

final_response_text = "Agent did not produce a final response."



async def process_response(prompt):

    content = types.Content(role='user', parts=[types.Part(text=prompt)])

    async for event in runner.run_async(user_id=USER_ID, session_id=SESSION_ID, new_message=content):
        if event.is_final_response():
            if event.content and event.content.parts:
                global final_response_text
                final_response_text = event.content.parts[0].text

await process_response(prompt)

display(Markdown(final_response_text))


I can help with that!  To give you the best possible plan, could you please tell me your name and preferred travel dates?  Also, what is your budget and what kind of experiences are you interested in (e.g., museums, nightlife, food tours, etc.)?


In [22]:
prompt = """
name - Velu
Travel date - August 4,
Budget - 100000 INR,
Travel style - budget friendly,
interests - nightlife,
travel companions - none,
"""

final_response_text = "Agent did not produce a final response."

await process_response(prompt)

display(Markdown(final_response_text))

Where would you like to depart from and what is your return date?


In [23]:
prompt = """
i will be departing from Mumbai and I would like to return after a week from my departure.
prepare the travel plan with this information. you can skip remaining questions
"""

final_response_text = "Agent did not produce a final response."

await process_response(prompt)

display(Markdown(final_response_text))

Here's a possible flight option based on your specifications:

* **Airline:** Emirates
* **Departure Time (Mumbai):** 7:00 AM
* **Arrival Time (Paris):** 12:00 PM
* **Price:** 30000 INR

Please note that this is just one example, and other options may be available depending on flight availability at the time of booking.  To proceed with booking, please provide your preferred dates and I can check for more options.  I will then pass this information to the HotelAgent to find suitable accommodations.
